In [ ]:
#Declare variables
import datetime
import pandas as pd
import numpy as np
#Constants - Storage Cost, Units in Millions

MILLION = 10**6
df = pd.read_csv('D:\Power BI thing\Forage Data sets\JPMC Data analyst\At_Gas.csv',index_col=0,parse_dates=True)
df_input = pd.read_csv('D:\Power BI thing\Forage Data sets\JPMC Data analyst\Contract Details.csv')

#To extract price at a specific date
X = df.values
#print(X)
#print(df)

#Injection and withdrawal rates
specific_date = '2021-01-31' # Replace with whatever input date you want to check

#Scaling the Rates as per volume

class Contract():
    def __init__(self,qty_bought,specific_date,transaction_type,storage_cost,injection_cost,max_volume,df):
        self.qty_bought = qty_bought
        self.specific_date = specific_date
        self.transaction_type = transaction_type
        self.storage_cost = storage_cost
        self.injection_cost = injection_cost
        self.max_volume = max_volume
        self.df = df

    def specific_rate(self):
        rate = []
        # Convert the specific date to a datetime object
        for i in range(0,len(self.specific_date)):
            valuetobeappenedintorate = self.df.loc[self.df.index == self.specific_date[i], 'Prices'].values
            if valuetobeappenedintorate.size > 0:
                
                rate.append(valuetobeappenedintorate[0])
            else: rate.append(0)

        return rate

    def price_calculation(self,current_rate):
        prices = []
        # print(self.qty_bought)
        # print(current_rate)
        # print(len(self.qty_bought))
        for i in range(0,len(self.qty_bought)):
            prices.append(int(self.qty_bought[i] * current_rate[i]))
            # if self.transaction_type[i] == 'I':
            #     prices[i] = prices[i] * (-1)
        return prices
    
    def inflow_outflow(self,buying_cost):
        for i in range(0, len(self.transaction_type)):
            if self.transaction_type[i] == 'I':
                buying_cost[i] = buying_cost[i] * (-1)
        return buying_cost
    
    def storage_running(self):
        #Max storage = 6
        #Current storage = 4 + 3 = 7(New Inj Qty)
        current_storage = [self.qty_bought[0]]
        for i in range (0,len(self.qty_bought)):
            if self.transaction_type[i] == 'W':
                self.qty_bought[i] = self.qty_bought[i] * (-1)
            #For 1st transaction, Current storage will be equal to First Qty_bought,
            #Append for all other cases
            if i != 0:
                current_storage.append(self.qty_bought[i]) #[i-1] because previous stored amount +/- new qty
            #If this is more than max 7 > 6 = True
            running_total = sum(current_storage)
            if running_total > self.max_volume:
                #Update the qty bought until maxstorage capacity is reached
                #New qty = 6 - 7 + 3
                self.qty_bought[i] = self.max_volume - current_storage[i] + self.qty_bought[i]
                # Storage is full
                current_storage[i] = self.max_volume
            elif running_total < 0:
                #If storage goes in n-ve, selling more than available , then empty all
                self.qty_bought[i] = current_storage[i]
                current_storage[i] = 0
        return current_storage

    def calc_storage_cost(self,storage_running):
        for i in range(0,len(storage_running)):
            storage_running[i] = abs((storage_running[i] * self.storage_cost))
        return storage_running
    
    def conv_to_mill(self,value):
        if isinstance(value,list):
            for i in range (0,len(value)):
                value[i] = value[i] * MILLION
            return value
        elif isinstance(value,(int,float)):
            return value*MILLION
    
    def inject_withdraw_cost(self):
        inj_cost = []
        for i in range(0,len(self.qty_bought)):
            inj_cost.append(abs((self.qty_bought[i] * self.injection_cost)))
        return inj_cost

count_rows = len(df.index)

qty_bought = df_input['Qty'].values
dates = df_input['Dates'].values
transaction_type = df_input['Transaction type'].values
print(qty_bought,dates,transaction_type)


def main():

    contract = Contract(qty_bought,dates,transaction_type,10000,10000,10,df)
    running_tally = contract.storage_running()
    print(running_tally,"running tally")
    cal_storage = contract.calc_storage_cost(running_tally)
    print(cal_storage,"total storage cost")
    current_rate = contract.specific_rate()
    print(current_rate,"current Rate")
    buying_cost = contract.price_calculation(current_rate)
    #buying_cost = contract.inflow_outflow(buying_cost)
    print(buying_cost, "buying cost")
    inj_cost = contract.inject_withdraw_cost()
    print(inj_cost,"inj_cost")

    total_buying_cost = sum(buying_cost)
    total_storage_cost = sum(cal_storage)
    total_inj_withdra_cost = sum(inj_cost)

    print(total_buying_cost + total_storage_cost + total_inj_withdra_cost)

    return

main()


[1.  1.5 4.  4. ] ['31-01-2021' '30-11-2021' '30-06-2022' '31-01-2023'] ['I' 'W' 'I' 'W']
[np.float64(1.0), 0, np.float64(4.0), np.float64(-4.0)] running tally
[np.float64(10000.0), 0, np.float64(40000.0), np.float64(40000.0)] total storage cost
[np.float64(10.9), np.float64(11.2), np.float64(10.4), np.float64(12.1)] current Rate
[10, -16, 41, -48] buying cost
[np.float64(10000.0), np.float64(15000.0), np.float64(40000.0), np.float64(40000.0)] inj_cost
194987.0


C:\Users\Nikhil\AppData\Local\Temp\ipykernel_15128\1313487049.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('D:\Power BI thing\Forage Data sets\JPMC Data analyst\At_Gas.csv',index_col=0,parse_dates=True)
